In [25]:
# To zhuzhu

import re
import requests
import pandas as pd
import numpy as np
from time import sleep
from bs4 import BeautifulSoup
import pickle # for loading a dictionary from disk
from typing import Optional # typehint that value can also be None
from dataprep.clean import validate_country
from dataprep.clean import clean_country


import json,re,os
from urllib.parse import quote

In [20]:
#!pip install dataprep

In [21]:
import spacy
from spacy import displacy
import warnings
warnings.filterwarnings("ignore")

# Larger and slower pipeline but more accurate
# spacy.cli.download("en_core_web_trf")
# NER= spacy.load("en_core_web_trf")
                   
# Faster and smaller pipelien, less accurate                   
spacy.cli.download("en_core_web_sm")
NER= spacy.load("en_core_web_sm")

def extract_country_from_abstract(abs_string):
    text_ner = NER(abs_string)
    #displacy.render(text_ner,style="ent",jupyter=True)

    # Extract the text from each tag
    tag_list = np.array([X.text for X in text_ner.ents])
 #   tag_list_df = pd.DataFrame({ "country": tag_list })
  #  tag_list_clean = clean_country(tag_list_df, "country").country_clean.values
    GPE_mask = np.array([X.label_ =='GPE'  for X in text_ner.ents])
    LOC_mask= np.array([X.label_ =='LOC'  for X in text_ner.ents])
    mask = [validate_country(tag) for tag in tag_list]

    # Abstract_valid_country
    abs_country = tag_list[mask]
    #abs_GPE 
    GPE_list,LOC_list = [],[]
    if len(GPE_mask)>0:
      GPE_list = tag_list[GPE_mask]
    if len(LOC_mask)>0:
      LOC_list = tag_list[LOC_mask]
    return abs_country, GPE_list, LOC_list


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [41]:
def read_page(info_url,abs_url):
    res = pd.DataFrame(columns= ['DOI','Journal','Type','Title','Publish_Time','Issue_Time','Co Author','Author','Rank','Nationality','Abs_country','Abs_GPE','Abs_LOC'])
    headers = {'user-agent': '(Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36'}
    home_page = requests.get(info_url, headers=headers)

    raw_html = home_page.text
    soup = BeautifulSoup(raw_html, 'html.parser')

    dl = soup.find('body').find('section',{'class':'article-body content authors'}).find('dl')
    authors = [dt.getText().split('\n')[1] for dt in dl.find_all('dt')]
    contributors_details = dl.find_all('dd')

    header = soup.find('body').find('main').find('div').find('header')
    Type = header.find('div',{'class':'article-meta'}).find('div',{'class':'article-type'}).find('p').getText()
    Title_etc = header.find('div',{'class':'article-title-etc'})
    doi = Title_etc.find('ul',recursive=False).find('a').getText()
    time = Title_etc.find('ul',recursive=False).find('li').getText().split('Published: ')[-1]
    Title = Title_etc.find('div',{'class':'title-authors'}).find('h1').getText()
    res['Author'] = authors
    res['DOI']= doi
    res['Journal']= 'PLOS ONE'
    res['Type']= Type
    res['Title']= Title
    res['Publish_Time']= time
    res['Issue_Time']= time
    res['Co Author'] = len(contributors_details) > 1

          
    # Extract the abstract
    home_page = requests.get(abs_url, headers=headers)
    raw_html = home_page.text
    soup_abs = BeautifulSoup(raw_html, 'html.parser')
    abs_text = soup_abs.find('div',{'class':'article-content'}).find('div',{'class':'abstract-content'}).find('p').get_text()
    abs_country, GPE_list, LOC_list = extract_country_from_abstract(abs_text)

    for author_describe_idx in range(len(contributors_details)):
        temp_dict={}
        affiliation_num = 'authAffiliations-'+str(author_describe_idx)
        detail = contributors_details[author_describe_idx]
        if detail.find('p',{'id':affiliation_num})!= None:
            affiliation_des = detail.find('p',{'id':affiliation_num}).getText()
            res.loc[author_describe_idx,'Nationality'] = affiliation_des.split(',')[-1].strip(' \n')
        else:
            res.loc[author_describe_idx,'Nationality'] = np.nan
        if author_describe_idx == 0 :
            res.loc[author_describe_idx,'Rank'] = 'First'
        else:
            res.loc[author_describe_idx,'Rank'] = 'Other'
        print(abs_country,GPE_list,LOC_list)
        res.at[author_describe_idx,'Abs_country'] = abs_country
        res.at[author_describe_idx,'Abs_GPE'] = GPE_list
        res.at[author_describe_idx,'Abs_LOC'] = LOC_list
    return res

In [23]:
def read_url_df(url_df):
    res = pd.DataFrame(columns= ['DOI','Journal','Type','Title','Publish_Time','Issue_Time','Co Author','Author','Rank','Nationality','Abs_country','Abs_GPE','Abs_LOC'])
    i=1
    for (info_url,abs_url) in url_df.values:
        df = read_page(info_url,abs_url)
        res = res.append(df)
        print(f'reading the {i}th paper, 937 in total')
        i+=1
        
    return res

In [ ]:
def get_url_list():
    def read_one(url):
        headers = {'user-agent': '(Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36'}
        base_url = 'https://journals.plos.org/plosone/article/authors?id='
        abs_base_url = 'https://journals.plos.org/plosone/article?id='
        home_page = requests.get(url, headers=headers)
        raw_html = home_page.text
        soup = BeautifulSoup(raw_html, 'html.parser')
        body = soup.find('body').find('main').find('div',{'id':'search-results-block'}).find('div',{'id':'subject-cover-view'})
        info_list = body.find('div',{'id':'subject-list-view'}).find_all('ul')
        info_url_lists = [base_url + info.find('h2').find('a').get('href').split('id=')[-1] for info in info_list]
        abs_url_lists = [abs_base_url + info.find('h2').find('a').get('href').split('id=')[-1] for info in info_list]
        return info_url_lists, abs_url_lists
    home_lists = ['https://journals.plos.org/plosone/browse/archaeology?resultView=list&page='+str(i) for i in range(1,74)]
    # Flatten the list
    tuple_list = [read_one(url) for url in home_lists]
    info_final =[]
    abs_final = []
    for inf,Abs in tuple_list:
        info_final.append(inf)
        abs_final.append(Abs)
    return sum(info_final,[]), sum(abs_final,[])

In [ ]:
info_final,abs_final =get_url_list()
url_df = pd.DataFrame({'info':info_final,'abs':abs_final})
url_df.to_csv('url.csv')

In [10]:
url_df= pd.read_csv('url.csv').drop(columns = ['Unnamed: 0'])

array([['https://journals.plos.org/plosone/article/authors?id=10.1371/journal.pone.0265640',
        'https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0265640'],
       ['https://journals.plos.org/plosone/article/authors?id=10.1371/journal.pone.0267163',
        'https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0267163'],
       ['https://journals.plos.org/plosone/article/authors?id=10.1371/journal.pone.0266146',
        'https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0266146'],
       ...,
       ['https://journals.plos.org/plosone/article/authors?id=10.1371/journal.pone.0001596',
        'https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0001596'],
       ['https://journals.plos.org/plosone/article/authors?id=10.1371/journal.pone.0001529',
        'https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0001529'],
       ['https://journals.plos.org/plosone/article/authors?id=10.1371/journal.pone.0000248',


In [42]:
plos_df = read_url_df(url_df)

流式输出内容被截断，只能显示最后 5000 行内容。
['Chile'] ['Chile' 'Patagonia'] []
['Chile'] ['Chile' 'Patagonia'] []
['Chile'] ['Chile' 'Patagonia'] []
['Chile'] ['Chile' 'Patagonia'] []
reading the 335th paper, 937 in total
['Sweden'] ['Sweden' 'δ15N'] []
['Sweden'] ['Sweden' 'δ15N'] []
['Sweden'] ['Sweden' 'δ15N'] []
reading the 336th paper, 937 in total
['136'] [] []
['136'] [] []
reading the 337th paper, 937 in total
['Italy' 'Italy' '90' 'SV' 'SV' 'SV' 'Italy'] ['Italy' 'Verona' 'Italy' 'hypoplasia' 'SV' 'hypoplasia' 'SV' 'Italy'] []
['Italy' 'Italy' '90' 'SV' 'SV' 'SV' 'Italy'] ['Italy' 'Verona' 'Italy' 'hypoplasia' 'SV' 'hypoplasia' 'SV' 'Italy'] []
['Italy' 'Italy' '90' 'SV' 'SV' 'SV' 'Italy'] ['Italy' 'Verona' 'Italy' 'hypoplasia' 'SV' 'hypoplasia' 'SV' 'Italy'] []
['Italy' 'Italy' '90' 'SV' 'SV' 'SV' 'Italy'] ['Italy' 'Verona' 'Italy' 'hypoplasia' 'SV' 'hypoplasia' 'SV' 'Italy'] []
reading the 338th paper, 937 in total
[] ['ZooArchNet'] []
[] ['ZooArchNet'] []
[] ['ZooArchNet'] []
[] ['ZooArchNe

In [43]:
plos_df.to_csv('PLOS.csv')

In [44]:
plos_df.shape

(6080, 13)

In [45]:
plos_df

,DOI,Journal,Type,Title,Publish_Time,Issue_Time,Co Author,Author,Rank,Nationality,Abs_country,Abs_GPE,Abs_LOC
0,https://doi.org/10.1371/journal.pone.0265640,PLOS ONE,Research Article,Mechanical properties of lithic raw materials ...,"April 22, 2022","April 22, 2022",True,Abay Namen,First,Kazakhstan,[],[the Inner Asian Mountain Corridor],[the Palaeolithic of Kazakhstan]
1,https://doi.org/10.1371/journal.pone.0265640,PLOS ONE,Research Article,Mechanical properties of lithic raw materials ...,"April 22, 2022","April 22, 2022",True,Radu Iovita,Other,United States of America,[],[the Inner Asian Mountain Corridor],[the Palaeolithic of Kazakhstan]
2,https://doi.org/10.1371/journal.pone.0265640,PLOS ONE,Research Article,Mechanical properties of lithic raw materials ...,"April 22, 2022","April 22, 2022",True,Klaus G. Nickel,Other,Germany,[],[the Inner Asian Mountain Corridor],[the Palaeolithic of Kazakhstan]
3,https://doi.org/10.1371/journal.pone.0265640,PLOS ONE,Research Article,Mechanical properties of lithic raw materials ...,"April 22, 2022","April 22, 2022",True,Aristeidis Varis,Other,Germany,[],[the Inner Asian Mountain Corridor],[the Palaeolithic of Kazakhstan]
4,https://doi.org/10.1371/journal.pone.0265640,PLOS ONE,Research Article,Mechanical properties of lithic raw materials ...,"April 22, 2022","April 22, 2022",True,Zhaken Taimagambetov,Other,Kazakhstan,[],[the Inner Asian Mountain Corridor],[the Palaeolithic of Kazakhstan]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,https://doi.org/10.1371/journal.pone.0000248,PLOS ONE,Research Article,Melanesian mtDNA Complexity,"February 28, 2007","February 28, 2007",True,George Koki,Other,Papua New Guinea,"[32, 16, Australia, Taiwan]","[Australia, Taiwan]","[Island Southeast Asia, Island Southeast Asia/..."
5,https://doi.org/10.1371/journal.pone.0000248,PLOS ONE,Research Article,Melanesian mtDNA Complexity,"February 28, 2007","February 28, 2007",True,Gisele Horvat,Other,United States of America,"[32, 16, Australia, Taiwan]","[Australia, Taiwan]","[Island Southeast Asia, Island Southeast Asia/..."
6,https://doi.org/10.1371/journal.pone.0000248,PLOS ONE,Research Article,Melanesian mtDNA Complexity,"February 28, 2007","February 28, 2007",True,Sergey Zhadanov,Other,United States of America,"[32, 16, Australia, Taiwan]","[Australia, Taiwan]","[Island Southeast Asia, Island Southeast Asia/..."
7,https://doi.org/10.1371/journal.pone.0000248,PLOS ONE,Research Article,Melanesian mtDNA Complexity,"February 28, 2007","February 28, 2007",True,Theodore G. Schurr,Other,United States of America,"[32, 16, Australia, Taiwan]","[Australia, Taiwan]","[Island Southeast Asia, Island Southeast Asia/..."
